**HTML 코드 구조 확인**

In [23]:
import requests
from bs4 import BeautifulSoup
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
response = requests.get(url, verify=False)  # SSL 인증 우회
soup = BeautifulSoup(response.content, "html.parser")

# 페이지의 전체 구조 확인
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   | 한국은행 기준금리 추이(목록) | 통화정책방향 | 통화정책 | 정책/업무 |  한국은행 홈페이지
  </title>
  <meta content="IE=edge" http-equiv="X-UA-Compatible">
   <meta content="기준금리,추이,변동,기준금리 그래프" name="kwrd">
    <meta content="한국은행에서 발표하는 대한민국 기준금리의 추이 정보 제공" name="dc"/>
    <meta content="한국은행 기준금리 추이" property="title"/>
    <meta content="한국은행" property="main"/>
    <meta content="" property="creator"/>
    <meta content="한국은행" property="publisher"/>
    <meta content=" | 한국은행 기준금리 추이(목록) | 통화정책방향 | 통화정책 | 정책/업무 |  한국은행 홈페이지" property="subject"/>
    <meta content="기준금리,추이,변동,기준금리 그래프" property="keyword"/>
    <meta content="한국은행에서 발표하는 대한민국 기준금리의 추이 정보 제공" property="description"/>
    <link href="https://static-cdn.bok.or.kr/static/commons/bok.ico" rel="icon" type="image/x-icon"/>
    <link href="https://static-cdn.bok.or.kr/static/portal/css/sub.css" rel="styleshee

**ECB에서 원/유로 환율 가져오기**

In [2]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_eur_krw_from_ecb_xml():
    # ECB XML 데이터 URL
    url = "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/krw.xml"
    
    try:
        response = requests.get(url, verify=False)  # SSL 인증 우회
        response.raise_for_status()  # HTTP 에러 체크
    except requests.exceptions.RequestException as e:
        print("HTTP 요청 중 문제가 발생했습니다:", e)
        raise

    # XML 파싱
    root = ET.fromstring(response.content)
    namespace = {"ns": "http://www.ecb.europa.eu/vocabulary/stats/exr/1"}  # XML 네임스페이스

    # 데이터 추출
    observations = root.findall(".//ns:Obs", namespace)
    if not observations:
        raise ValueError("XML 데이터에서 <Obs> 태그를 찾을 수 없습니다.")

    data = []
    for obs in observations:
        date = obs.get("TIME_PERIOD")  # 날짜
        rate = obs.get("OBS_VALUE")   # 환율
        if date and rate:
            data.append([date, float(rate)])

    # DataFrame 생성
    df = pd.DataFrame(data, columns=["Date", "EUR/KRW"])
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.set_index("Date", inplace=True)
    
    return df

# 정상동작 여부 테스트
if __name__ == "__main__":
    eur_krw_data = fetch_eur_krw_from_ecb_xml()
    print(eur_krw_data.head())
    print("\n최근 데이터:")
    print(eur_krw_data.tail())


            EUR/KRW
Date               
1999-01-04  1398.59
1999-01-05  1373.01
1999-01-06  1359.54
1999-01-07  1337.16
1999-01-08  1366.73

최근 데이터:
            EUR/KRW
Date               
2025-06-30  1588.21
2025-07-01  1598.18
2025-07-02  1598.57
2025-07-03  1604.80
2025-07-04  1604.30


# 여기가 본게임 (준비물 : USD/EUR, USD/KRW) #

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from xml.etree import ElementTree as ET
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Fetch EUR/KRW exchange rate data from ECB XML
def fetch_eur_krw_from_ecb_xml():
    url = "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/krw.xml"
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("HTTP 요청 중 문제가 발생했습니다:", e)
        raise

    root = ET.fromstring(response.content)
    namespace = {"ns": "http://www.ecb.europa.eu/vocabulary/stats/exr/1"}

    observations = root.findall(".//ns:Obs", namespace)
    if not observations:
        raise ValueError("XML 데이터에서 <Obs> 태그를 찾을 수 없습니다.")

    eur_krw_data = []  # Renamed from 'data' for clarity
    for obs in observations:
        date = obs.get("TIME_PERIOD")
        rate = obs.get("OBS_VALUE")
        if date and rate:
            eur_krw_data.append([date, float(rate)])

    eur_krw_df = pd.DataFrame(eur_krw_data, columns=["Date", "EUR/KRW"])  # Avoid generic 'df'
    eur_krw_df["Date"] = pd.to_datetime(eur_krw_df["Date"], format="%Y-%m-%d")
    eur_krw_df.set_index("Date", inplace=True)
    return eur_krw_df

# Fetch Federal Reserve data
def fetch_fed_rate():
    csv_url = "https://www.federalreserve.gov/aboutthefed/files/target-funds-2014-2024.csv"
    try:
        response = requests.get(csv_url, verify=False)
        response.raise_for_status()
        csv_data = StringIO(response.text)
        fed_rate_data = pd.read_csv(csv_data)  # Renamed from 'df' to avoid conflict
        latest_date = fed_rate_data.columns[-1]
        low_rate = fed_rate_data.loc[0, latest_date]
        high_rate = fed_rate_data.loc[1, latest_date]
        fed_rate = (low_rate + high_rate) / 2
        return fed_rate
    except Exception as e:
        print(f"Error fetching Fed rate: {e}")
        return None


# Fetch Bank of Korea data
def fetch_bok_rate():
    url_bok = "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
    try:
        response_bok = requests.get(url_bok, verify=False)
        response_bok.raise_for_status()
        soup_bok = BeautifulSoup(response_bok.text, 'html.parser')
        table_bok = soup_bok.find('table', class_='fixed')
        latest_row_bok = table_bok.find_all('tr')[1]
        cols_bok = latest_row_bok.find_all('td')
        bok_rate = float(cols_bok[2].text.strip())
        return bok_rate
    except Exception as e:
        print(f"Error fetching BOK rate: {e}")
        return None

# Fetch ECB rate data
def fetch_ecb_rate():
    url_ecb = "https://data.ecb.europa.eu/data-detail-transform/FM.D.U2.EUR.4F.KR.MRR_FR.LEV/143"
    try:
        response_ecb = requests.get(url_ecb, verify=False)
        response_ecb.raise_for_status()
        data_ecb = response_ecb.json()
        observation = data_ecb[0]["observationList"][0]
        ecb_rate = float(observation["obsValue"])
        return ecb_rate
    except Exception as e:
        print(f"Error fetching ECB rate: {e}")
        return None

# Calculate today’s EUR/KRW rate
def calculate_today_eur_krw(usd_eur, usd_krw):
    return usd_krw / usd_eur

# IRP-adjusted EUR/KRW
def calculate_irp_eur_krw(usd_eur, usd_krw, fed_rate, bok_rate, ecb_rate):
    usd_krw_irp = usd_krw * (1 + (bok_rate / 100)) / (1 + (fed_rate / 100))
    eur_krw_irp = usd_krw_irp / usd_eur
    return eur_krw_irp

# Calculate EUR/KRW from USD/EUR and USD/KRW
def calculate_today_eur_krw(usd_eur, usd_krw):
    return usd_krw / usd_eur

# Calculate IRP-adjusted EUR/KRW
def calculate_irp_eur_krw(fed_rate, bok_rate, ecb_rate, usd_eur, usd_krw):
    eur_krw = usd_krw / usd_eur
    irp_adjustment = (1 + ecb_rate / 100) / (1 + bok_rate / 100)
    return eur_krw * irp_adjustment

# Analyze trends and moving averages
def analyze_trend_and_bounds(data):
    data["MA5"] = data["EUR/KRW"].rolling(window=5).mean()
    data["MA20"] = data["EUR/KRW"].rolling(window=20).mean()
    data["MA60"] = data["EUR/KRW"].rolling(window=60).mean()

    recent_data = data[-60:]
    lower_support = recent_data["EUR/KRW"].min()
    upper_resistance = recent_data["EUR/KRW"].max()

    recent_rate = data["EUR/KRW"].iloc[-1]
    ma5 = data["MA5"].iloc[-1]
    ma20 = data["MA20"].iloc[-1]
    ma60 = data["MA60"].iloc[-1]

    # 이격도 계산
    gap_ma5_ma20 = (ma5 - ma20) / ma20 * 100  # 단기-중기 이격도 (%)

    # 추세 판단 로직 수정
    if recent_rate > ma5 > ma20 > ma60 and gap_ma5_ma20 > 0.5:
        trend = "급등 추세"
    elif recent_rate < ma5 < ma20 < ma60 and gap_ma5_ma20 < -0.5:
        trend = "급락 추세"
    elif recent_rate > ma5 and recent_rate > ma20:
        trend = "상승 추세"
    elif recent_rate < ma5 and recent_rate < ma20 and recent_rate < ma60:
        trend = "하락 추세"
    else:
        trend = "조정 국면"

    print(f"이동평균선 (5일): {ma5:.2f}")
    print(f"이동평균선 (20일): {ma20:.2f}")
    print(f"이동평균선 (60일): {ma60:.2f}")
    print(f"이격도 (MA5-MA20): {gap_ma5_ma20:.2f}%")

    return trend, lower_support, upper_resistance

# Calculate spread
def calculate_spread(today_eur_krw, lower_support, upper_resistance):
    spread_lower = max(lower_support, today_eur_krw * 0.998)
    spread_upper = min(upper_resistance, today_eur_krw * 1.002)
    return spread_lower, spread_upper


def main():
    # Fetch rates
    fed_rate = fetch_fed_rate()
    bok_rate = fetch_bok_rate()
    ecb_rate = fetch_ecb_rate()
    
    if None in (fed_rate, bok_rate, ecb_rate):
        print("필요한 기준금리 데이터를 확인할 수 없습니다. Fetch하는 경로를 점검하세요.")
        return

    print(f"Fed Rate: {fed_rate:.2f}%, BOK Rate: {bok_rate:.2f}%, ECB Rate: {ecb_rate:.2f}%")

    try:
        usd_eur = float(input("현재 USD/EUR 환율을 입력하세요: "))
        usd_krw = float(input("현재 USD/KRW 환율을 입력하세요: "))
    except ValueError:
        print("환율 입력값이 유효하지 않습니다. 숫자를 입력하세요.")
        return

    # Fetch historical EUR/KRW data
    try:
        eur_krw_data = fetch_eur_krw_from_ecb_xml()
    except Exception as e:
        print(f"Error fetching EUR/KRW data: {e}")
        return

    # Calculate exchange rates
    today_eur_krw_unadjusted = calculate_today_eur_krw(usd_eur, usd_krw)
    today_eur_krw_adjusted = calculate_irp_eur_krw(fed_rate, bok_rate, ecb_rate, usd_eur, usd_krw)

    # Analyze trends and calculate bounds
    trend, lower_support, upper_resistance = analyze_trend_and_bounds(eur_krw_data)
    spread_lower, spread_upper = calculate_spread(today_eur_krw_unadjusted, lower_support, upper_resistance) #스프레드 (±20bp 변동)에 spread_lower ~ spread_upper를 표기해주나,
    #일일 스프레드가 20bp 이상일 때는 유효하지 않으므로 today_eur_krw_unadjusted에서 수기로 적용할 것 (e.g. ±30bp: [lower] *0.997, [upper] *1.003)

    print("\n--- 분석 완료 ---")
    print(f"\n오늘 계산된 EUR/KRW 적정가 (IRP 조정 전): {today_eur_krw_unadjusted:.2f}")
    print(f"오늘 계산된 EUR/KRW 적정가 (IRP 조정 후): {today_eur_krw_adjusted:.2f}")
    print(f"현재 추세: {trend}")
    print(f"하방저지선: {lower_support:.2f}, 상방저지선: {upper_resistance:.2f}")
    print(f"스프레드 (±10bp 변동): {today_eur_krw_unadjusted * 0.999:.2f} ~ {today_eur_krw_unadjusted * 1.001:.2f}")
    print(f"스프레드 (±20bp 변동): {today_eur_krw_unadjusted * 0.998:.2f} ~ {today_eur_krw_unadjusted * 1.002:.2f}")

if __name__ == "__main__":
    main()

Fed Rate: 4.12%, BOK Rate: 2.50%, ECB Rate: 2.15%
이동평균선 (5일): 1631.04
이동평균선 (20일): 1625.71
이동평균선 (60일): 1614.96
이격도 (MA5-MA20): 0.33%

--- 분석 완료 ---

오늘 계산된 EUR/KRW 적정가 (IRP 조정 전): 1636.78
오늘 계산된 EUR/KRW 적정가 (IRP 조정 후): 1631.19
현재 추세: 상승 추세
하방저지선: 1587.13, 상방저지선: 1633.10
스프레드 (±10bp 변동): 1635.15 ~ 1638.42
스프레드 (±20bp 변동): 1633.51 ~ 1640.06


**Fed에서 금리 땡겨오기**

In [3]:
import pandas as pd
import requests
from io import StringIO
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# URL for the CSV file
csv_url = "https://www.federalreserve.gov/aboutthefed/files/target-funds-2014-2024.csv"

try:
    # Fetch the CSV file with SSL verification disabled
    response = requests.get(csv_url, verify=False)
    response.raise_for_status()  # Raise an error for bad responses

    # Parse the CSV content
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)

    # Display the first few rows of the dataframe
    print("Federal Funds Rate Data:")
    print(df.head())

except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching the CSV file: {e}")

# Find the most recent column (last date)
latest_date = df.columns[-1]

# Calculate the middle value (average of low and high rates for the most recent date)
low_rate = df.loc[0, latest_date]
high_rate = df.loc[1, latest_date]
fed_int = (low_rate + high_rate) / 2

print(f"Most recent date: {latest_date}")
print(f"Federal Reserve Board Funds Rate (MEDIAN): {fed_int}")

Federal Funds Rate Data:
   2014-03-20T00:00:00.000Z  2014-05-01T00:00:00.000Z  \
0                      0.00                      0.00   
1                      0.25                      0.25   

   2014-06-19T00:00:00.000Z  2014-07-31T00:00:00.000Z  \
0                      0.00                      0.00   
1                      0.25                      0.25   

   2014-09-18T00:00:00.000Z  2014-10-30T00:00:00.000Z  \
0                      0.00                      0.00   
1                      0.25                      0.25   

   2014-12-18T00:00:00.000Z  2015-01-29T00:00:00.000Z  \
0                      0.00                      0.00   
1                      0.25                      0.25   

   2015-03-19T00:00:00.000Z  2015-04-30T00:00:00.000Z  ...  3/21/2024  \
0                      0.00                      0.00  ...       5.25   
1                      0.25                      0.25  ...       5.50   

   5/2/2024  6/13/2024  8/1/2024  9/19/2024  11/8/2024  12/19/2024 

**BOK에서 금리 땡겨오기**

In [4]:
import requests
from bs4 import BeautifulSoup
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Step 1: Get the page content
url = "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
response = requests.get(url, verify=False)  # SSL 인증 우회
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract the most recent table row
table = soup.find('table', class_='fixed')
latest_row = table.find_all('tr')[1]  # Skip the header row, and pick the most recent row

# Step 3: Extract the most recent date and interest rate
cols = latest_row.find_all('td')
bok_decision_date = f"{cols[0].text.strip()} {cols[1].text.strip()}"
bok_rate = float(cols[2].text.strip())

# Print the results
print(f"Most recent decision date: {bok_decision_date}")
print(f"Most recent base rate: {bok_rate}%")

Most recent decision date: 2025 02월 25일
Most recent base rate: 2.75%


**종합선물세트**
*Fed에서 금리 땡겨오기*
*BOK에서 금리 땡겨오기*
*ECB에서 금리 땡겨오기*

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Fetch Federal Reserve rate
def fetch_fed_rate():
    csv_url = "https://www.federalreserve.gov/aboutthefed/files/target-funds-2014-2024.csv"
    try:
        response = requests.get(csv_url, verify=False)
        response.raise_for_status()
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data)
        latest_date = df.columns[-1]
        low_rate = df.loc[0, latest_date]
        high_rate = df.loc[1, latest_date]
        return (low_rate + high_rate) / 2
    except Exception as e:
        print(f"Error fetching Federal Reserve rate: {e}")
        return None

# Fetch Bank of Korea rate
def fetch_bok_rate():
    url_bok = "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
    try:
        response_bok = requests.get(url_bok, verify=False)
        response_bok.raise_for_status()
        soup_bok = BeautifulSoup(response_bok.text, 'html.parser')
        table_bok = soup_bok.find('table', class_='fixed')
        latest_row_bok = table_bok.find_all('tr')[1]
        cols_bok = latest_row_bok.find_all('td')
        return float(cols_bok[2].text.strip())
    except Exception as e:
        print(f"Error fetching Bank of Korea rate: {e}")
        return None

# Fetch ECB rate
def fetch_ecb_rate():
    url_ecb = "https://data.ecb.europa.eu/data-detail-transform/FM.D.U2.EUR.4F.KR.MRR_FR.LEV/143"
    try:
        response_ecb = requests.get(url_ecb, verify=False)
        response_ecb.raise_for_status()
        data_ecb = response_ecb.json()
        observation = data_ecb[0]["observationList"][0]
        return float(observation["obsValue"])
    except Exception as e:
        print(f"Error fetching ECB rate: {e}")
        return None

# Calculate EUR/KRW using IRP
def calculate_irp_eur_krw(usd_eur, usd_krw, fed_rate, bok_rate, ecb_rate):
    # USD/KRW adjusted for BOK and Fed rates
    usd_krw_irp = usd_krw * (1 + bok_rate / 100) / (1 + fed_rate / 100)
    # EUR/KRW adjusted for ECB and BOK rates
    eur_krw_irp = (usd_krw_irp / usd_eur) * (1 + ecb_rate / 100) / (1 + bok_rate / 100)
    return eur_krw_irp

# Main script
def main():
    # Fetch rates
    fed_rate = fetch_fed_rate()
    bok_rate = fetch_bok_rate()
    ecb_rate = fetch_ecb_rate()

    if None in (fed_rate, bok_rate, ecb_rate):
        print("Error fetching central bank rates.")
        return

    print(f"Fed Rate: {fed_rate:.2f}%, BOK Rate: {bok_rate:.2f}%, ECB Rate: {ecb_rate:.2f}%")

    try:
        usd_eur = float(input("Enter current USD/EUR exchange rate: "))
        usd_krw = float(input("Enter current USD/KRW exchange rate: "))
    except ValueError:
        print("Invalid exchange rate input. Please enter numeric values.")
        return

    # Calculate adjusted EUR/KRW rate
    irp_eur_krw = calculate_irp_eur_krw(usd_eur, usd_krw, fed_rate, bok_rate, ecb_rate)
    print(f"IRP-adjusted EUR/KRW rate: {irp_eur_krw:.2f}")

if __name__ == "__main__":
    main()


Fed Rate: 4.62%, BOK Rate: 3.00%, ECB Rate: 3.40%
IRP-adjusted EUR/KRW rate: 1489.79


In [9]:
%pip install matplotlib

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------------------------------- -- 7.3/7.8 MB 93.5 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 26.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 62.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 90.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
